<a href="https://colab.research.google.com/github/AndrewCrossman/Graduate_Research/blob/main/BNL_Scripts/Convert_tth_to_xy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Convert_tth_to_xy**

*This code converts .chi files located in a google drive directory into .xy files*

In [1]:
from google.colab import drive  #allows downloading and accessing files on drive
from google.colab import files  #allows downloading and accessing files on drive
from datetime import datetime   #new object datetime
import shutil                   #allows uploading files to drive
import os                       #allows deleting files from drive
import glob                     #useful for collecting and organizing files
import csv
import numpy as np              #math library
import pandas as pd             #data organization library
import scipy as sp              #math library
import re                       #string processing library
from scipy import optimize      #math fitting library
import matplotlib.pylab as plt  #allows for plotting
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['plt', 'datetime']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [2]:
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
directory_path = '/content/drive/Shareddrives/BNL_June_2024/DataFiles/Cu1Ni2_200C/integration'
tth_files = glob.glob(directory_path + '/*tth.chi')
print(len(tth_files))

2195


In [ ]:
data = pd.read_csv(tth_files[0])
print(data.shape[0])  # Gives number of rows
print(data.shape[1])  # Gives number of columns
data

3000
1


,# chi_2theta chi_I
0,2.891794622277297014e-03 1.561303257942199707e+00
1,8.675383866831891042e-03 8.798288702964782715e-01
2,1.445897311138648507e-02 4.799991250038146973e-01
3,2.024256235594108083e-02 2.137174367904663086e+00
4,2.602615160049567139e-02 2.070718765258789062e+00
...,...
2995,1.732474158206328951e+01 -1.010480523109436035...
2996,1.733052517130784054e+01 2.106397002935409546e-01
2997,1.733630876055239511e+01 8.241237401962280273e-01
2998,1.734209234979694969e+01 1.506945788860321045e-01


In [ ]:
for file in tth_files:
  xy_name = file.replace(".chi",".xy")
  data = pd.read_csv(file, skiprows=1, header=None, sep="\s+").astype(float)
  data.to_csv(xy_name, header=False, sep='\t', index=False)  #downloads csv file directly to drive with no header, tab separations, and no index
  #print(xy_name)

## How to Delete Files in Google Drive Using Colab

In [ ]:
# my all files starts with "frame" and ends with ".jpg"
    fileList = glob.glob('/content/drive/MyDrive/frame*.jpg')
    print("Number of files: ",len(fileList))

    for filePath in fileList:
        try:
            os.remove(filePath)
        except:
            print("Error while deleting file : ", filePath)

## Resequence File Names with timestamp

In [5]:
directory_path = '/content/drive/Shareddrives/BNL_June_2024/DataFiles/Cu1Ni2_200C/integration/'
tth_files = glob.glob(directory_path + '/*tth.xy')
len(tth_files)

2195

In [6]:
date_format = '%Y%m%d%H%M%S'
file_data = pd.DataFrame(columns=['file_name','datetime','seq_num'])

for file in tth_files:
  file_numbers = re.findall(r'\d+',file)[7:]
  date_obj = datetime.datetime.strptime(file_numbers[0]+file_numbers[1], date_format)
  file_data.loc[len(file_data.index)] = [file, date_obj,float(file_numbers[-1])]

file_data.sort_values(['datetime','seq_num']).reset_index(drop=True)

,file_name,datetime,seq_num
0,/content/drive/Shareddrives/BNL_June_2024/Data...,2024-06-11 22:43:49,1.0
1,/content/drive/Shareddrives/BNL_June_2024/Data...,2024-06-11 22:54:45,1.0
2,/content/drive/Shareddrives/BNL_June_2024/Data...,2024-06-11 22:56:07,1.0
3,/content/drive/Shareddrives/BNL_June_2024/Data...,2024-06-11 22:56:07,2.0
4,/content/drive/Shareddrives/BNL_June_2024/Data...,2024-06-11 22:56:07,3.0
...,...,...,...
2190,/content/drive/Shareddrives/BNL_June_2024/Data...,2024-06-11 23:27:53,360.0
2191,/content/drive/Shareddrives/BNL_June_2024/Data...,2024-06-11 23:27:53,361.0
2192,/content/drive/Shareddrives/BNL_June_2024/Data...,2024-06-11 23:27:53,362.0
2193,/content/drive/Shareddrives/BNL_June_2024/Data...,2024-06-11 23:27:53,363.0


In [ ]:
for file in tth_files:
  file_numbers = re.findall(r'\d+',file)[7:]
  date_obj = datetime.datetime.strptime(file_numbers[0]+file_numbers[1], date_format)
  print(file,date_obj, float(file_numbers[-1]))
  real_seq_num = file_data.index[(file_data['datetime']==date_obj) & (file_data['seq_num'] == float(file_numbers[-1]))].tolist()[0]
  new_name = directory_path+"{}_{}_Seq_num{}_Real_seq_num{}.xy".format(file_numbers[0], file_numbers[1], file_numbers[-1],real_seq_num)
  data = pd.read_csv(file, header=None, sep="\s+").astype(float)
  data.to_csv(new_name, header=False, sep='\t', index=False)  #downloads csv file directly to drive with no header, tab separations, and no index

/content/drive/Shareddrives/BNL_June_2024/DataFiles/Cu1Ni2_200C/integration/Cu1Ni2_mixed_200_20240611-225607_275e1a_primary-158_mean_tth.xy 2024-06-11 22:56:07 158.0
/content/drive/Shareddrives/BNL_June_2024/DataFiles/Cu1Ni2_200C/integration/Cu1Ni2_mixed_200_20240611-225607_275e1a_primary-160_mean_tth.xy 2024-06-11 22:56:07 160.0
/content/drive/Shareddrives/BNL_June_2024/DataFiles/Cu1Ni2_200C/integration/Cu1Ni2_mixed_200_20240611-225607_275e1a_primary-155_mean_tth.xy 2024-06-11 22:56:07 155.0
/content/drive/Shareddrives/BNL_June_2024/DataFiles/Cu1Ni2_200C/integration/Cu1Ni2_mixed_200_20240611-225607_275e1a_primary-154_mean_tth.xy 2024-06-11 22:56:07 154.0
/content/drive/Shareddrives/BNL_June_2024/DataFiles/Cu1Ni2_200C/integration/Cu1Ni2_mixed_200_20240611-225607_275e1a_primary-153_mean_tth.xy 2024-06-11 22:56:07 153.0
/content/drive/Shareddrives/BNL_June_2024/DataFiles/Cu1Ni2_200C/integration/Cu1Ni2_mixed_200_20240611-225607_275e1a_primary-165_mean_tth.xy 2024-06-11 22:56:07 165.0
/con